Notebook Settings
=================

``` ipython
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

%run ../notebooks/setup.py
%matplotlib inline
%config InlineBackend.figure_format = 'png'
```

Imports
=======

``` ipython
import sys
sys.path.insert(0, '../')

import torch
import gc
import pandas as pd
from time import perf_counter

from src.network import Network
from src.plot_utils import plot_con
from src.decode import decode_bump, circcvl
from src.utils import clear_cache
```

Helpers
=======

``` ipython
def convert_seconds(seconds):
    h = seconds // 3600
    m = (seconds % 3600) // 60
    s = seconds % 60
    return h, m, s
```

``` ipython
import numpy as np
def get_theta(a, b, GM=0, IF_NORM=0):

    u, v = a, b

    if GM:
        v = b - np.dot(b, a) / np.dot(a, a) * a

    if IF_NORM:
        u = a / np.linalg.norm(a)
        v = b / np.linalg.norm(b)

    return np.arctan2(v, u)
```

``` ipython
def get_idx(model):
    ksi = model.PHI0.cpu().detach().numpy()
    theta = get_theta(ksi[0], ksi[2], GM=0, IF_NORM=0)
    return theta.argsort()
```

``` ipython
def get_overlap(model, rates):
    ksi = model.PHI0.cpu().detach().numpy()
    return rates @ ksi.T / rates.shape[-1]

```

``` ipython
def get_correct_perf(rates, d_phi, n_ini=50):
    m0, m1, phi = decode_bump(rates, axis=-1)
    x = m1[..., -1] / m0[..., -1] * np.cos(phi[..., -1] - d_phi * np.pi / 180)
    performance = (x[:n_ini] < 0).mean() * 100
    performance += (x[n_ini:] > 0).mean() * 100

    return performance / 2
```

Dual task with rotating readout
===============================

Parameters
----------

``` ipython
REPO_ROOT = "/home/leon/models/NeuroFlame"
conf_name = "config_EI.yml"
```

Fixed Points
------------

### Simulation

``` ipython
seed = np.random.randint(10000)
# 34, 53
print(seed)
```

``` ipython
n_ini  = 1000

LR_COV = [[1.0, 1.0, 0.0, 0.0],
          [0.8, 1.0, 0.0, 0.0],
          [0.0, 0.0, 1.0, 0.8],
          [0.0, 0.0, 0.8, 1.0]]

start = perf_counter()
model = Network(conf_name, REPO_ROOT, VERBOSE=0, DEVICE='cuda:1',
                N_BATCH=2*n_ini, SEED=seed, LR_COV=LR_COV,
                TASK='dual_rand', LIVE_FF_UPDATE=1, DURATION=10)

rates_fp = model(REC_LAST_ONLY=1).detach().cpu().numpy()
end = perf_counter()

print("Elapsed (with compilation) = %dh %dm %ds" % convert_seconds(end - start))
print('rates', rates_fp.shape)
```

### Readout

``` ipython
Ne = model.Na[0].detach().cpu().numpy()
N = model.N_NEURON
```

``` ipython
idx = get_idx(model)
rates_ordered = rates_fp[:, idx]
print(rates_ordered.shape)
```

``` ipython
m0, m1, phi = decode_bump(rates_ordered, axis=-1)
print(m0.shape)
```

``` ipython
phi0 = model.phase.cpu().numpy()[:, 0]
print(phi0.shape)
error = phi * 180 / np.pi - phi0
print(error.shape)
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=[2*width, height])
ax[0].hist(phi0 * 180 / np.pi, bins='auto', density=True)
ax[0].set_xlabel('Cue Loc. (°)')

ax[1].hist(phi * 180 / np.pi, bins='auto', density=True)
ax[1].set_xlabel('Error (°)')
plt.show()
```

``` ipython
idx = phi0.argsort()
plt.plot(phi0[idx] * 180.0 / np.pi, circcvl(error[idx], 200), '-')
plt.xlabel('Cue Loc. (°)')
plt.ylabel('Error (°)')
plt.show()
```

``` ipython
```

``` ipython
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2)

x = m1 / m0 * np.cos(phi)
y = m1 / m0 * np.sin(phi)

data = np.stack((x,y)).T
print(data.shape)

kmeans.fit(data)
centers = kmeans.cluster_centers_
print("Cluster Centers:", centers)
x_c1, y_c1 = centers[0]
x_c2, y_c2 = centers[1]

phi1 = np.arctan2(y_c1, x_c1)
phi2 = np.arctan2(y_c2, x_c2)
```

``` ipython
radius = np.sqrt(x_c1**2 + y_c1**2)

# Midpoint M of A and B
xM, yM = (x_c1 + x_c2) / 2, (y_c1 + y_c2) / 2
print(xM)

phi0 = ( (phi1 + phi2) / 2 ) % np.pi
psi0 = (phi0 + np.pi) % (2 * np.pi)

xPhi = radius * np.cos(phi0)
yPhi = radius * np.sin(phi0)

dist = np.sqrt((xPhi - xM)**2 + (yPhi - yM)**2)

xPsi = radius * np.cos(psi0)
yPsi = radius * np.sin(psi0)

dist = [np.sqrt((xPhi - xM)**2 + (yPhi - yM)**2),
        np.sqrt((xPsi - xM)**2 + (yPsi - yM)**2)]

print('dist', dist)

if dist[0]>dist[1]:
   mean_phi = phi0 * 180 / np.pi
else:
   mean_phi = psi0 * 180 / np.pi

print('mean phi', mean_phi)

```

``` ipython
fig, ax = plt.subplots(1, 1, figsize=[height, height])

x = m1 / m0 * np.cos(phi)
y = m1 / m0 * np.sin(phi)

ax.plot(x, y, 'ob', alpha=.25)

x = 1.5 * np.cos(mean_phi * np.pi / 180)
y = 1.5 * np.sin(mean_phi * np.pi / 180)

# Draw an arrow using plt.arrow(x, y, dx, dy, **kwargs)
plt.arrow(0, 0, x, y, head_width=0.5, head_length=0.5, fc='blue')

mean = np.array([mean_phi, mean_phi - 180 ]) * np.pi/180

x = 2 * np.cos(mean)
y = 2 * np.sin(mean)
ax.plot(x, y, '--b')

ax.plot(x_c1, y_c1, 'xk', alpha=1, ms=20)
ax.plot(x_c2, y_c2, 'xk', alpha=1, ms=20)

ax.set_xlim([-2, 2])
ax.set_ylim([-2, 2])

plt.show()
```

``` ipython
```

Single trial
------------

``` ipython
n_ini=100
model = Network(conf_name, REPO_ROOT, VERBOSE=0, DEVICE='cuda:1',
                N_BATCH=2*n_ini, SEED=seed, LR_COV=LR_COV,
                TASK='dual', LIVE_FF_UPDATE=1, DURATION=10,
                I0=[.2, .2])
```

``` ipython
start = perf_counter()
rates = model().detach().cpu().numpy()
end = perf_counter()
print("Elapsed (with compilation) = %dh %dm %ds" % convert_seconds(end - start))
print(rates.shape)
```

``` ipython
overlap = get_overlap(model, rates)
print(overlap.T.shape)

idx = get_idx(model)

rates_ordered = rates[..., idx]
print(rates_ordered.shape)

m0, m1, phi = decode_bump(rates_ordered, axis=-1)
print(m0.shape)
```

``` ipython
fig, ax = plt.subplots(1, 3, figsize=[2*width, height])

ax[0].plot(overlap.T[0])
ax[0].set_ylabel('Overlap on $m_S$ (Hz)')
ax[0].set_xlabel('Step')

ax[1].plot(overlap.T[1])
ax[1].set_ylabel('Overlap on $n_S$ (Hz)')
ax[1].set_xlabel('Step')

ax[2].plot(overlap.T[2])
ax[2].set_ylabel('Overlap on $m_D$ (Hz)')
ax[2].set_xlabel('Step')

plt.show()
```

``` ipython
fig, ax = plt.subplots(1, 3, figsize=[2*width, height])

ax[0].plot(m0.T)
#ax[0].set_ylim([0, 360])
#ax[0].set_yticks([0, 90, 180, 270, 360])
ax[0].set_ylabel('$\mathcal{F}_0$ (Hz)')
ax[0].set_xlabel('Step')

ax[1].plot(m1.T)
# ax[1].set_ylim([0, 360])
# ax[1].set_yticks([0, 90, 180, 270, 360])
ax[1].set_ylabel('$\mathcal{F}_1$ (Hz)')
ax[1].set_xlabel('Step')

ax[2].plot(phi.T * 180 / np.pi, alpha=0.25)
ax[2].set_ylim([0, 360])
ax[2].set_yticks([0, 90, 180, 270, 360])
ax[2].set_ylabel('Phase (°)')
ax[2].set_xlabel('Step')

plt.show()
```

``` ipython
fig, ax = plt.subplots(figsize=(height, height))

x = m1 / m0 * np.cos(phi)
y = m1 / m0 * np.sin(phi)

x = x[:2]
y = y[:2]

ax.plot(x.T[0], y.T[0], 'ob', alpha=.25, ms=10)
ax.plot(x.T, y.T, '-b', alpha=.5)
ax.plot(x.T[-1], y.T[-1], 'sb', alpha=.25, ms=10)

x = m1 / m0 * np.cos(phi)
y = m1 / m0 * np.sin(phi)

x = x[-2:]
y = y[-2:]
ax.plot(x.T[0], y.T[0], 'or', alpha=.25, ms=10)
ax.plot(x.T, y.T, '-r', alpha=.5)
ax.plot(x.T[-1], y.T[-1], 'sr', alpha=.25, ms=10)

mean = np.array([mean_phi, mean_phi - 180 ]) * np.pi/180

x = 2 * np.cos(mean)
y = 2 * np.sin(mean)
ax.plot(x, y, '--b')
x = 1.5 * np.cos(mean_phi * np.pi / 180)
y = 1.5 * np.sin(mean_phi * np.pi / 180)

# Draw an arrow using plt.arrow(x, y, dx, dy, **kwargs)
plt.arrow(0, 0, x, y, head_width=0.5, head_length=0.5, fc='blue')


ax.plot(x_c1, y_c1, 'xk', alpha=1, ms=20)
ax.plot(x_c2, y_c2, 'xk', alpha=1, ms=20)

ax.set_xlim([-2, 2])
ax.set_ylim([-2, 2])
plt.show()
```

``` ipython
fig, ax = plt.subplots(figsize=(height, height))

x = m1[:n_ini, -1] / m0[:n_ini, -1] * np.cos(phi[:n_ini, -1])
y = m1[:n_ini, -1] / m0[:n_ini, -1] * np.sin(phi[:n_ini, -1])

ax.plot(x, y, 'ob', alpha=.5)

x = m1[n_ini:, -1] / m0[n_ini:, -1] * np.cos(phi[n_ini:, -1])
y = m1[n_ini:, -1] / m0[n_ini:, -1] * np.sin(phi[n_ini:, -1])

ax.plot(x, y, 'or', alpha=.5)

mean = np.array([mean_phi, mean_phi - 180 ]) * np.pi/180

x = 2 * np.cos(mean)
y = 2 * np.sin(mean)
ax.plot(x, y, '--')

x = 1.5 * np.cos(mean_phi * np.pi / 180)
y = 1.5 * np.sin(mean_phi * np.pi / 180)

# Draw an arrow using plt.arrow(x, y, dx, dy, **kwargs)
plt.arrow(0, 0, x, y, head_width=0.5, head_length=0.5, fc='blue')

# mean = np.array([mean_phi - d_phi , mean_phi - d_phi - 180]) * np.pi/180
# x = 2 * np.cos(mean)
# y = 2 * np.sin(mean)

ax.plot(x_c1, y_c1, 'xk', alpha=1, ms=20)
ax.plot(x_c2, y_c2, 'xk', alpha=1, ms=20)

ax.set_xlim([-2, 2])
ax.set_ylim([-2, 2])

plt.show()
```

``` ipython
readoutA = overlap[:n_ini, -5:, 0]
readoutB = overlap[n_ini:, -5:, 0]

readout = np.stack((readoutA, readoutB))
print(readout.shape)
```

``` ipython
perf = (readout[0]>0).mean((0, 1))
perf += (readout[1]<0).mean((0,1))

print('performance', perf/2 * 100)
```

``` ipython
print(mean_phi-90)
```

``` ipython
perf = get_correct_perf(rates_ordered, mean_phi, n_ini)
print('corrected performance', perf)
```

``` ipython
```

Corrected trial
---------------

``` ipython
model = Network(conf_name, REPO_ROOT, VERBOSE=0, DEVICE='cuda:1',
                N_BATCH=2*n_ini, SEED=seed, LR_COV=LR_COV,
                TASK='dual_odr', LIVE_FF_UPDATE=1, DURATION=10,
                I0=[.2, 2])

model.PHI1 = torch.tensor([0, 180-mean_phi], device='cuda:1')
```

``` ipython
start = perf_counter()
rates = model().detach().cpu().numpy()
end = perf_counter()
print(rates.shape)
```

``` ipython
overlap = get_overlap(model, rates)
print(overlap.T.shape)

idx = get_idx(model)

rates_ordered = rates[..., idx]
print(rates_ordered.shape)

m0, m1, phi = decode_bump(rates_ordered, axis=-1)
print(m0.shape)
```

``` ipython
fig, ax = plt.subplots(1, 3, figsize=[2*width, height])

ax[0].plot(overlap.T[0])
ax[0].set_ylabel('Overlap on $m_S$ (Hz)')
ax[0].set_xlabel('Step')

ax[1].plot(overlap.T[1])
ax[1].set_ylabel('Overlap on $n_S$ (Hz)')
ax[1].set_xlabel('Step')

ax[2].plot(overlap.T[2])
ax[2].set_ylabel('Overlap on $m_D$ (Hz)')
ax[2].set_xlabel('Step')

plt.show()
```

``` ipython
fig, ax = plt.subplots(1, 3, figsize=[2*width, height])

ax[0].plot(m0.T)
#ax[0].set_ylim([0, 360])
#ax[0].set_yticks([0, 90, 180, 270, 360])
ax[0].set_ylabel('$\mathcal{F}_0$ (Hz)')
ax[0].set_xlabel('Step')

ax[1].plot(m1.T)
# ax[1].set_ylim([0, 360])
# ax[1].set_yticks([0, 90, 180, 270, 360])
ax[1].set_ylabel('$\mathcal{F}_1$ (Hz)')
ax[1].set_xlabel('Step')

ax[2].plot(phi.T * 180 / np.pi, alpha=0.25)
ax[2].axhline((180 - mean_phi) % 360, 0 , 100, ls='--', color='k')

ax[2].set_ylim([0, 360])
ax[2].set_yticks([0, 90, 180, 270, 360])
ax[2].set_ylabel('Phase (°)')
ax[2].set_xlabel('Step')

plt.show()
```

``` ipython
fig, ax = plt.subplots(figsize=(height, height))
x = m1 / m0 * np.cos(phi)
y = m1 / m0 * np.sin(phi)

x = x[:10]
y = y[:10]

ax.plot(x.T[0], y.T[0], 'ob', alpha=.25, ms=10)
ax.plot(x.T, y.T, '-b', alpha=.25)
ax.plot(x.T[-1], y.T[-1], 'sb', alpha=.25, ms=10)

x = m1 / m0 * np.cos(phi)
y = m1 / m0 * np.sin(phi)

x = x[-10:]
y = y[-10:]

ax.plot(x.T[0], y.T[0], 'or', alpha=.25, ms=10)
ax.plot(x.T, y.T, '-r', alpha=.25)
ax.plot(x.T[-1], y.T[-1], 'sr', alpha=.25, ms=10)

mean = np.array([mean_phi, mean_phi - 180 ]) * np.pi/180

x = 2 * np.cos(mean)
y = 2 * np.sin(mean)
ax.plot(x, y, '--b')

x = 1.5 * np.cos(mean_phi * np.pi / 180)
y = 1.5 * np.sin(mean_phi * np.pi / 180)

# Draw an arrow using plt.arrow(x, y, dx, dy, **kwargs)
plt.arrow(0, 0, x, y, head_width=0.5, head_length=0.5, fc='blue', alpha=.2)


ax.plot(x_c1, y_c1, 'xk', alpha=1, ms=20)
ax.plot(x_c2, y_c2, 'xk', alpha=1, ms=20)

ax.set_xlim([-2, 2])
ax.set_ylim([-2, 2])
plt.show()
```

``` ipython
fig, ax = plt.subplots(figsize=(height, height))

x = m1[:n_ini, -1] / m0[:n_ini, -1] * np.cos(phi[:n_ini, -1])
y = m1[:n_ini, -1] / m0[:n_ini, -1] * np.sin(phi[:n_ini, -1])

ax.plot(x, y, 'ob', alpha=.5)

x = m1[n_ini:, -1] / m0[n_ini:, -1] * np.cos(phi[n_ini:, -1])
y = m1[n_ini:, -1] / m0[n_ini:, -1] * np.sin(phi[n_ini:, -1])

ax.plot(x, y, 'or', alpha=.5)

mean = np.array([mean_phi, mean_phi - 180 ]) * np.pi/180

x = 2 * np.cos(mean)
y = 2 * np.sin(mean)
ax.plot(x, y, '--')

x = 1.5 * np.cos(mean_phi * np.pi / 180)
y = 1.5 * np.sin(mean_phi * np.pi / 180)

# Draw an arrow using plt.arrow(x, y, dx, dy, **kwargs)
plt.arrow(0, 0, x, y, head_width=0.5, head_length=0.5, fc='blue', alpha=.2)


ax.plot(x_c1, y_c1, 'xk', alpha=1, ms=20)
ax.plot(x_c2, y_c2, 'xk', alpha=1, ms=20)

ax.set_xlim([-2, 2])
ax.set_ylim([-2, 2])

plt.show()
```

``` ipython
readoutA = overlap[:n_ini, -5:, 0]
readoutB = overlap[n_ini:, -5:, 0]

readout = np.stack((readoutA, readoutB))
print(readout.shape)
```

``` ipython
perf = (readout[0]>0).mean((0, 1))
perf += (readout[1]<0).mean((0,1))

print('performance', perf/2 * 100)
```

``` ipython
print(mean_phi + 90)
```

``` ipython
perf = get_correct_perf(rates_ordered, (180-mean_phi), n_ini)
print('corrected performance', perf)
```

``` ipython
perf = get_correct_perf(rates_ordered, (180-mean_phi), n_ini)
print('corrected performance', perf)
```

``` ipython
```